In [91]:
%matplotlib inline
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import math
import numpy as np
import pandas as pd
import pathlib
import six
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
pd.set_option("display.width", 1000)
pd.set_option("display.max_rows", 50)
plt.style.use("ggplot")
plt.rcParams["font.size"] = 13
mpl.rcParams["font.family"] = "Osaka"

## Data

In [12]:
data_path = pathlib.Path("./DataPS201901.csv")

In [13]:
df = pd.read_csv(data_path, header=None, names=["choice", "age", "gender"])
df

,choice,age,gender
0,3,46,1
1,0,29,1
2,1,27,0
3,3,36,0
4,3,39,0
5,3,54,1
6,3,42,0
7,0,23,1
8,2,35,0
9,3,58,0


In [14]:
car_spec = np.array([[0, 0], [1, 5], [1.2, 3.5], [1.4, 2]])

## Estimation

The utility of player $i$ when choosing car $j$ is given by
\begin{align}
u_{ij} = \beta_1 \cdot age_i \cdot hp_j + \beta_2 \cdot gender_i \cdot fe_j + \epsilon_{ij}
\end{align}
where $\epsilon_{ij} \overset{\text{i.i.d.}}{\sim} F$, $F$ is Type I extreme distribution. Note that $\beta_0$ is deleted since it is common for all $j$.

Then the log likelihood function is given by
\begin{align}
\ell(\mathbf{\beta}) \equiv \sum_i \sum_j y_{ij} \log P_{ij} = \sum_i \sum_j y_{ij} \left( \beta^\text{T} x_{ij} - \log \left(\sum_{k=0}^3 \exp(\beta^\text{T} x_{ik}) \right) \right)
\end{align}
where $P_{ij} \equiv \exp(\beta^\text{T} x_{ij}) / \sum_{k=0}^3 \exp(\beta^\text{T} x_{ik})$.

Since $\ell(\mathbf{\beta})$ is concave in $\beta$ (I will show the concavity in the last page), I can obtain the global maximum by the simple Newton method.


In [101]:
# choice, agent, beta_index
covariates = np.empty([4, df.shape[0], 2], dtype=float)
covariates[:, :, :] = car_spec[:, None, :]
covariates[:, :, 0] *= df["age"][None, :]
covariates[:, :, 1] *= df["gender"][None, :]
covariates

array([[[ 0. ,  0. ],
        [ 0. ,  0. ],
        [ 0. ,  0. ],
        ...,
        [ 0. ,  0. ],
        [ 0. ,  0. ],
        [ 0. ,  0. ]],

       [[46. ,  5. ],
        [29. ,  5. ],
        [27. ,  0. ],
        ...,
        [56. ,  0. ],
        [34. ,  0. ],
        [32. ,  5. ]],

       [[55.2,  3.5],
        [34.8,  3.5],
        [32.4,  0. ],
        ...,
        [67.2,  0. ],
        [40.8,  0. ],
        [38.4,  3.5]],

       [[64.4,  2. ],
        [40.6,  2. ],
        [37.8,  0. ],
        ...,
        [78.4,  0. ],
        [47.6,  0. ],
        [44.8,  2. ]]])

In [102]:
y = np.identity(4)[df["choice"]].T

def obj_fun(beta):
    beta_x = covariates @ beta
    inside_par = beta_x - np.log(np.sum(np.exp(beta_x), axis=0))
    return -1 * np.sum(y * inside_par, axis=None)


In [107]:
beta_init = np.array([1.0, 1.0])
result = opt.minimize(obj_fun, beta_init, method="BFGS")
result

      fun: 8522.264404907895
 hess_inv: array([[ 0.   , -0.   ],
       [-0.   ,  0.001]])
      jac: array([0.003, 0.   ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 264
      nit: 13
     njev: 63
   status: 2
  success: False
        x: array([ 0.072, -1.328])

The estimates are $(\hat{\beta}_1^{\text{MLE}}, \hat{\beta}_2^{\text{MLE}}) = (0.072, -1.328)$